# Remove fiducials: Run this after organzing dot files (dots_comb created)

In [ ]:
from remove_fiducials import *

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        name = "Gaussian"
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name = "LoG"
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False, add_trace = True, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        if len(tiff.shape) == 3:
            tiff = tiff.reshape(1,tiff.shape[0],tiff.shape[1],tiff.shape[2])
        print("shape =", tiff.shape)
        #plot
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)

In [ ]:
#test on one pos
locations = "/groups/CaiLab/personal/Lex/raw/150genes_040122/notebook_pyfiles/dots_comb/final-thresh0/Channel_1/MMStack_Pos0/locations_z_0.csv"
fiducial_path = "/groups/CaiLab/personal/Lex/raw/150genes_040122/Fiducials/MMStack_Pos0.ome.tif"

In [ ]:
#remove fiducials
removed, fid_locs = remove_all_fiducials(locations, fiducial_path, threshold=500, 
                                         radius=1, swapaxes=True, write=False)

In [ ]:
#isolate single hyb
location_0 = removed[removed["hyb"]==0]

In [ ]:
#check to see if fiducials are picked
plot_3d_locs_on_2d_image(fid_locs,None, tiff=None, channel=0, raw_src = fiducial_path, 
                         raw_image = True, add_trace=False, zmax=2000)

In [ ]:
#check to see if those fiducial dots are removed from locations file
plot_3d_locs_on_2d_image(location_0,None, tiff=None, channel=0, raw_src = fiducial_path, 
                         raw_image = True, add_trace=False, zmax=2000)

In [ ]:
# make sure dots look normal on a hyb
hyb=0
pos=0
img_raw = f'/groups/CaiLab/personal/Lex/raw/150genes_040122/notebook_pyfiles/dapi_aligned/fiducial_aligned/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
plot_3d_locs_on_2d_image(location_0,None, tiff=None, channel=0, raw_src = img_raw, 
                         raw_image = True, add_trace=False, zmax=2000)